# 🏥 Radyoloji LLM Karşılaştırma

**🆓 Tüm modeller ücretsiz!**  
**📂 Google Drive'a kaydeder**

---

## 1️⃣ Kurulum

In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Projeyi GitHub'dan indir
!git clone https://github.com/Hanketsu3/LLMComparison.git /content/LLMComparison

import sys, os, gc
from PIL import Image

PROJECT_ROOT = '/content/LLMComparison'
os.chdir(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)
print(f"✅ PROJECT_ROOT: {PROJECT_ROOT}")

In [ ]:
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available(): print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
%pip install -q transformers>=4.40.0 accelerate bitsandbytes
%pip install -q Pillow pyyaml tqdm pandas scikit-learn evaluate rouge-score

## 2️⃣ Model Registry

In [ ]:
from src.utils import PromptManager, print_model_table
print_model_table()

In [ ]:
pm = PromptManager()
print("Promptlar:", pm.list_prompts())

## 3️⃣ Test Görüntüsü

In [ ]:
test_path = f"{PROJECT_ROOT}/src/img/1280px-Normal_posteroanterior_(PA)_chest_radiograph_(X-ray).jpg"
test_image = Image.open(test_path).convert("RGB")
print(f"📷 {test_image.size}")

## 4️⃣ Qwen2-VL-2B Testi

In [ ]:
%%time
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor

MODEL = "Qwen/Qwen2-VL-2B-Instruct"
print(f"🔄 {MODEL}...")
processor = AutoProcessor.from_pretrained(MODEL)
model = Qwen2VLForConditionalGeneration.from_pretrained(MODEL, torch_dtype=torch.float16, device_map="auto")
print("✅ Yüklendi!")

In [ ]:
def generate_report(image, prompt_text):
    messages = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": prompt_text}]}]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True).to("cuda")
    output_ids = model.generate(**inputs, max_new_tokens=512)
    return processor.batch_decode(output_ids, skip_special_tokens=True)[0]

In [ ]:
%%time
prompt = pm.get_prompt("rrg", "detailed").user_prompt
report = generate_report(test_image, prompt)
print("📋 RAPOR:\n" + "="*50 + "\n" + report)

## 5️⃣ Prompt Karşılaştırması

In [ ]:
prompt_results = {}
for name in ["baseline", "detailed", "turkish"]:
    print(f"🔄 {name}...")
    prompt_results[name] = generate_report(test_image, pm.get_prompt("rrg", name).user_prompt)
    print(f"✅ {len(prompt_results[name])} chars")

In [ ]:
for n, r in prompt_results.items():
    print(f"\n{'='*50}\n📋 {n.upper()}\n{'='*50}\n{r[:500]}...")

In [ ]:
del model, processor; gc.collect(); torch.cuda.empty_cache()
print(f"🧹 GPU: {torch.cuda.memory_allocated()/1e9:.1f} GB")

## 6️⃣ Sonuçları Kaydet (Drive'a)

In [ ]:
import json
from datetime import datetime

# Drive'da results klasörü oluştur
DRIVE_RESULTS = '/content/drive/MyDrive/LLMComparison_Results'
os.makedirs(DRIVE_RESULTS, exist_ok=True)

output_path = f"{DRIVE_RESULTS}/experiment_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
with open(output_path, "w") as f:
    json.dump({
        "timestamp": datetime.now().isoformat(),
        "model": "Qwen2-VL-2B",
        "prompt_comparison": prompt_results
    }, f, indent=2, ensure_ascii=False)

print(f"✅ Kaydedildi: {output_path}")
print("📂 Drive'da: My Drive > LLMComparison_Results")

---
## ✅ Tamamlandı!

Sonuçlar Google Drive'a kaydedildi.